In [ ]:
# Federated Learning - Section Project #2
# Central server is not trusted with dataset

# code sources:
# https://blog.openmined.org/encrypted-deep-learning-classification-with-pysyft/
# https://www.simonwenkel.com/2019/07/20/introduction-to-pysyft.html
# https://github.com/pytorch/examples/blob/master/mnist/main.py
'''
1. Train a dataset
2. Use federated learning
3. Gradients not come up to central learning in raw form
4. Use .move command to move gradients to one of the workers
5. Sum gradients at one worker 
6. Bring sum back to central server
Note: central server never sees gradient for one worker - one worker does see gradient
'''

In [3]:
# pytorch library
import torch

from torch.utils import data

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import copy

# pysyft library
import syft as sy

#argument parser
import argparse

W0728 22:43:35.123500  1584 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Claudia\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0728 22:43:35.149499  1584 deprecation_wrapper.py:119] From C:\Users\Claudia\Anaconda3\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [4]:
# Orchestrate movement of data from one virtual machine to another
# worker simulates interface we have to another machine
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
jon = sy.VirtualWorker(hook, id="jon")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [5]:
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=None)

9920512it [00:04, 2013908.29it/s]                                              


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz


32768it [00:00, 409600.39it/s]


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz


1654784it [00:01, 1547973.80it/s]                                              


Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz


8192it [00:00, 167183.59it/s]


Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [6]:
mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=None)

In [7]:
mnist_trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train

In [8]:
mnist_testset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../data
    Split: Test

In [ ]:
# simplified arguments - learning parameters
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 200
        self.epochs = 10
        self.lr = 0.001 # learning rate
        self.log_interval = 100

args = Arguments()

In [ ]:
# MNIST handwritten database of images, labels 
#training set images, training set labels
#test set images, test set labels

# regular train_loader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), 
    batch_size=args.batch_size, shuffle=True)

In [ ]:
#test_loader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

In [9]:
class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.load('..data/' + ID + '.pt')
        y = self.labels[ID]

        return X, y

In [ ]:
# CUDA for PyTorch
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda:0" if use_cuda else "cpu")
# cudnn.benchmark = True

# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 1}
max_epochs = 1
# max_epochs = 100

# Datasets
partition = 60000
labels = 10000

# data = torch.tensor(mnist_trainset, requires_grad=True)
# target = torch.tensor(mnist_testset, requires_grad=True)


# Generators
training_set = Dataset(partition['train'], labels)
data = data.DataLoader(training_set, **params)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['validation'], labels)
target = data.DataLoader(validation_set, **params)
validation_generator = data.DataLoader(validation_set, **params)

In [ ]:
len(mnist_trainset), len(mnist_testset)

In [ ]:
# get pointers to training data on each worker by
# sending some training data to bob and alice and jon

data = torch.tensor(mnist_trainset, requires_grad=True)
target = torch.tensor(mnist_testset, requires_grad=True)

#Data pointed to VirtualWorkers 
bobs_data = data.send(bob)
bobs_target = target.send(bob)

alices_data = data.send(alice)
alices_target = target.send(alice)

jons_data = data.send(jon)
jons_target = target.send(jon)

In [ ]:
# send copy of model to alice and bob

bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)
jons_model = model.copy().send(jon)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)
jons_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [ ]:
#train bob and alice and jon's model in parallel

for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    # Train Jon's Model
    jons_opt.zero_grad()
    jons_pred = jons_model(jons_data)
    jons_loss = ((jons_pred - jons_target)**2).sum()
    jons_loss.backward()

    jons_opt.step()
    jons_loss = jons_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss) +  + " Jon:" + str(jons_loss))

In [ ]:
# send all models to a secure worker
alices_model.move(secure_worker)
bobs_model.move(secure_worker)
jons_model.move(secure_worker)

In [ ]:
#average trained models together
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data + jons_model.weight.data) / 3).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data + jons_model.bias.data) / 3).get())

In [ ]:
# iterate a couple of times
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)
    jons_model = model.copy().send(jon)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)
    jons_opt = optim.SGD(params=jons_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
        
         # Train Jon's Model
        jons_opt.zero_grad()
        jons_pred = jons_model(jons_data)
        jons_loss = ((jons_pred - jons_target)**2).sum()
        jons_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    jons_model.move(secure_worker)
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data + jons_model.weight.data) / 3).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data + jons_model.bias.data) / 3).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss) + " Jon:" + str(jons_loss))

In [ ]:
#evalutate dataset
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [ ]:
print(preds)
print(target)
print(loss.data)